In [12]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.feature_extraction import DictVectorizer

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_score, recall_score, make_scorer
from sklearn.metrics import brier_score_loss

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate


import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm


import xml.etree.ElementTree as ET
import json

import Feature_Extraction as fe

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ga75xoh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
print(tf.__version__)

1.14.0


In [14]:
root = ET.parse('data/RTE1_dev1_3ways.xml').getroot()
r1 = []
r2 = []
y = []
for el in list(root):
    label = el.get('entailment')
    if label == 'YES':
        y.append(-1)
    elif label == 'NO':
        y.append(1)
    else:
        continue
    r1.append(list(el)[0].text)
    r2.append(list(el)[1].text)
    
root = ET.parse('data/RTE1_dev2_3ways.xml').getroot()
for el in list(root):
    label = el.get('entailment')
    if label == 'YES':
        y.append(-1)
    elif label == 'NO':
        y.append(1)
    else:
        continue
    r1.append(list(el)[0].text)
    r2.append(list(el)[1].text)
    
r1t = []
r2t = []
yt = []
root = ET.parse('data/RTE1_test_3ways.xml').getroot()
for el in list(root):
    label = el.get('entailment')
    if label == 'YES':
        yt.append(-1)
    elif label == 'NO':
        yt.append(1)
    else:
        continue
    r1t.append(list(el)[0].text)
    r2t.append(list(el)[1].text)

In [15]:
r1 = r1 + r1t[:365]
r2 = r2 + r2t[:365]
y = y + yt[:365]
r1t = r1t[-184:]
r2t = r2t[-184:]
yt = yt[-184:]

In [16]:
x = []
for i in range(len(y)):
    x.append(fe.createFBag(r1[i], r2[i]))

In [17]:
x_test = []
for i in range(len(yt)):
    x_test.append(fe.createFBag(r1t[i], r2t[i]))

In [18]:
vec = DictVectorizer()
xx = vec.fit_transform(x).toarray()
xx_test = vec.transform(x_test).toarray()

In [20]:
LR_clf =  LogisticRegression(C=4.281332398719396, class_weight='balanced', penalty= 'l1', solver= 'liblinear')
LR_clf.fit(xx, y)
y_pred = LR_clf.predict(xx_test)
print(classification_report(yt, y_pred))

              precision    recall  f1-score   support

          -1       0.73      0.82      0.77       125
           1       0.48      0.34      0.40        59

    accuracy                           0.67       184
   macro avg       0.60      0.58      0.58       184
weighted avg       0.65      0.67      0.65       184



In [45]:
xx.shape

(750, 599)

In [101]:
model = keras.Sequential()
model.add(keras.layers.Embedding(599, 128))
#model.add(keras.layers.Dense(units=20, activation='relu', input_shape=xx.shape))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(128, activation=tf.nn.relu))
model.add(keras.layers.Dense(64, activation=tf.nn.relu))
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, None, 128)         76672     
_________________________________________________________________
global_average_pooling1d_10  (None, 128)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_41 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_42 (Dense)             (None, 16)                1040      
_________________________________________________________________
dense_43 (Dense)             (None, 1)                 17        
Total params: 102,497
Trainable params: 102,497
Non-trainable params: 0
_______________________________________________

In [102]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [103]:
history = model.fit(xx,y)

750/750 [==============================] - ETA: 4s - loss: 0.6790 - acc: 0.0000e+0 - ETA: 1s - loss: 0.6406 - acc: 0.0000e+0 - ETA: 0s - loss: 0.6086 - acc: 0.0000e+0 - ETA: 0s - loss: 0.5739 - acc: 0.0000e+0 - ETA: 0s - loss: 0.5267 - acc: 0.0000e+0 - ETA: 0s - loss: 0.4877 - acc: 0.0000e+0 - ETA: 0s - loss: 0.4369 - acc: 0.0000e+0 - ETA: 0s - loss: 0.3857 - acc: 0.0000e+0 - 1s 798us/sample - loss: 0.3528 - acc: 0.0000e+00


In [104]:
results = model.evaluate(xx_test, yt)

results

184/184 [==============================] - ETA: 0s - loss: -0.4490 - acc: 0.0000e+ - 0s 661us/sample - loss: -0.1411 - acc: 0.0000e+00


[-0.1410935433662456, 0.0]